# Open AI मॉडेल्सचे फाइन ट्यूनिंग

हा नोटबुक Open AI च्या [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) डोक्युमेंटेशनमधील सध्याच्या मार्गदर्शनावर आधारित आहे.

फाइन-ट्यूनिंग म्हणजे बेस मॉडेलला तुमच्या अ‍ॅप्लिकेशनसाठी अधिक उपयुक्त बनवण्यासाठी, त्या विशिष्ट वापरासाठी किंवा परिस्थितीसाठी संबंधित अतिरिक्त डेटा आणि संदर्भासह पुन्हा ट्रेन करणे. लक्षात ठेवा, _few shot learning_ आणि _retrieval augmented generation_ सारख्या प्रॉम्प्ट इंजिनिअरिंग तंत्रांचा वापर करून तुम्ही प्रॉम्प्टमध्ये संबंधित डेटा घालून गुणवत्ता वाढवू शकता. मात्र, या पद्धतींना टार्गेट फाउंडेशन मॉडेलच्या कमाल टोकन विंडो साइजने मर्यादा असते.

फाइन-ट्यूनिंगमध्ये आपण प्रत्यक्षात मॉडेलला आवश्यक डेटासह पुन्हा ट्रेन करतो (यामुळे आपण कमाल टोकन विंडोमध्ये बसू शकणाऱ्या उदाहरणांपेक्षा बरेच जास्त उदाहरणे वापरू शकतो) - आणि एक _कस्टम_ मॉडेल तयार करतो ज्याला इनफरन्स वेळी उदाहरणे देण्याची गरज राहत नाही. यामुळे केवळ आपल्या प्रॉम्प्ट डिझाइनची कार्यक्षमता वाढत नाही (कारण टोकन विंडो इतर गोष्टींसाठी वापरण्याची अधिक लवचिकता मिळते) तर कदाचित खर्चही कमी होतो (कारण इनफरन्स वेळी मॉडेलला पाठवावे लागणारे टोकन्स कमी होतात).

फाइन ट्यूनिंगचे ४ टप्पे आहेत:
1. ट्रेनिंग डेटा तयार करा आणि अपलोड करा.
1. ट्रेनिंग जॉब चालवा आणि फाइन-ट्यून केलेले मॉडेल मिळवा.
1. फाइन-ट्यून केलेल्या मॉडेलचे मूल्यांकन करा आणि गुणवत्ता सुधारण्यासाठी प्रक्रिया पुन्हा करा.
1. समाधानकारक वाटल्यास, इनफरन्ससाठी फाइन-ट्यून केलेले मॉडेल डिप्लॉय करा.

सर्व फाउंडेशन मॉडेल्सना फाइन-ट्यूनिंग सपोर्ट नसतो - [OpenAI डोक्युमेंटेशन तपासा](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) ताज्या माहितीसाठी. तुम्ही पूर्वी फाइन-ट्यून केलेले मॉडेलही पुन्हा फाइन-ट्यून करू शकता. या ट्युटोरियलमध्ये आपण `gpt-35-turbo` हे फाइन-ट्यूनिंगसाठी टार्गेट फाउंडेशन मॉडेल म्हणून वापरणार आहोत.

---


### टप्पा 1.1: तुमचा डेटासेट तयार करा

चला एक असा चॅटबॉट तयार करूया जो तुम्हाला आवर्त सारणीतील घटकांबद्दल प्रश्न विचारल्यावर लायमरिकच्या स्वरूपात उत्तर देईल. _या_ सोप्या ट्युटोरियलमध्ये, आपण फक्त काही उदाहरणे वापरून डेटासेट तयार करू, ज्यात अपेक्षित डेटा स्वरूप दाखवणारी उत्तरे असतील. प्रत्यक्ष वापरात, तुम्हाला बरीच उदाहरणे असलेला डेटासेट तयार करावा लागेल. तुमच्या वापरासाठी योग्य असा ओपन डेटासेट उपलब्ध असेल, तर तो वापरून त्याचे स्वरूप बदलून फाइन-ट्यूनिंगसाठी वापरता येईल.

आपण `gpt-35-turbo` वर लक्ष केंद्रित करत आहोत आणि एकाच टप्प्यातील (single-turn) उत्तराची अपेक्षा करत आहोत (chat completion), म्हणून आपण [हे सुचवलेले स्वरूप](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) वापरून उदाहरणे तयार करू शकतो, जे OpenAI च्या chat completion च्या गरजा पूर्ण करते. जर तुम्हाला बहुपदरी (multi-turn) संवाद अपेक्षित असेल, तर तुम्ही [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) वापराल, ज्यात `weight` नावाचा parameter असतो, जो फाइन-ट्यूनिंग प्रक्रियेत कोणती संदेशे वापरायची हे ठरवतो.

या ट्युटोरियलसाठी आपण सोपे single-turn स्वरूप वापरणार आहोत. डेटा [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) मध्ये आहे, ज्यात प्रत्येक ओळीत एक रेकॉर्ड असतो, आणि प्रत्येक रेकॉर्ड JSON स्वरूपात असतो. खालील उदाहरणात 2 रेकॉर्ड दाखवले आहेत - संपूर्ण उदाहरण संचासाठी [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) पहा (आपण फाइन-ट्यूनिंग ट्युटोरियलसाठी 10 उदाहरणे वापरणार आहोत). **टीप:** प्रत्येक रेकॉर्ड _एकाच_ ओळीत असावा (सामान्य JSON फाईलप्रमाणे ओळींमध्ये विभागू नये)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

खऱ्या वापरात चांगले परिणाम मिळवण्यासाठी तुम्हाला बरेच मोठे उदाहरण संच लागेल - उत्तरेची गुणवत्ता आणि फाइन-ट्यूनिंगसाठी लागणारा वेळ/खर्च यामध्ये समतोल साधावा लागेल. आपण लहान संच वापरत आहोत जेणेकरून प्रक्रिया लवकर पूर्ण होईल आणि समजावता येईल. अधिक गुंतागुंतीच्या फाइन-ट्यूनिंग ट्युटोरियलसाठी [OpenAI Cookbook चे हे उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) पहा.


### टप्पा 1.2 आपला डेटासेट अपलोड करा

Files API वापरून डेटा अपलोड करा [येथे दिल्याप्रमाणे](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). लक्षात ठेवा, हे कोड चालवण्यासाठी तुम्ही आधी खालील गोष्टी केलेल्या असाव्यात:
 - `openai` Python पॅकेज इन्स्टॉल केलेले असावे (नवीन फीचर्ससाठी आवृत्ती >=0.28.0 वापरा)
 - `OPENAI_API_KEY` एन्व्हायर्नमेंट व्हेरिएबलमध्ये तुमची OpenAI API key सेट केलेली असावी
अधिक माहितीसाठी, या कोर्ससाठी दिलेल्या [सेटअप मार्गदर्शिका](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) पहा.

आता, आपल्या स्थानिक JSONL फाइलमधून अपलोडसाठी फाइल तयार करण्यासाठी कोड चालवा.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### टप्पा 2.1: SDK वापरून फाइन-ट्यूनिंग जॉब तयार करा


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### टप्पा 2.2: job ची स्थिती तपासा

`client.fine_tuning.jobs` API वापरून तुम्ही खालील गोष्टी करू शकता:
- `client.fine_tuning.jobs.list(limit=<n>)` - शेवटचे n फाइन-ट्यूनिंग jobs यादीत दाखवा
- `client.fine_tuning.jobs.retrieve(<job_id>)` - एखाद्या विशिष्ट फाइन-ट्यूनिंग job चे तपशील मिळवा
- `client.fine_tuning.jobs.cancel(<job_id>)` - फाइन-ट्यूनिंग job रद्द करा
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - job मधील जास्तीत जास्त n events यादीत दाखवा
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

या प्रक्रियेतील पहिला टप्पा म्हणजे _training file ची पडताळणी करणे_, म्हणजेच डेटा योग्य स्वरूपात आहे याची खात्री करणे.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### टप्पा 2.3: प्रगतीवर लक्ष ठेवण्यासाठी इव्हेंट्स ट्रॅक करा


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Step 2.4: OpenAI डॅशबोर्डमध्ये स्थिती पहा


तुम्ही OpenAI वेबसाइटवर जाऊन आणि प्लॅटफॉर्ममधील _Fine-tuning_ विभागात पाहूनही स्थिती तपासू शकता. यामुळे तुम्हाला चालू असलेल्या जॉबची स्थिती दिसेल, तसेच मागील जॉब एक्झिक्युशन रनचे इतिहासही पाहता येईल. या स्क्रीनशॉटमध्ये, तुम्ही पाहू शकता की मागील एक्झिक्युशन अयशस्वी झाले होते, आणि दुसरा रन यशस्वी झाला. संदर्भासाठी, हे तेव्हा घडले जेव्हा पहिल्या रनमध्ये चुकीच्या स्वरूपातील रेकॉर्ड्स असलेली JSON फाइल वापरली गेली होती - एकदा ती दुरुस्त केल्यावर, दुसरा रन यशस्वीपणे पूर्ण झाला आणि मॉडेल वापरासाठी उपलब्ध झाले.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.mr.png)


तुम्ही खालीलप्रमाणे दृश्य डॅशबोर्डमध्ये स्क्रोल करून स्टेटस संदेश आणि मेट्रिक्स देखील पाहू शकता:

| संदेश | मेट्रिक्स |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.mr.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.mr.png)|


### टप्पा 3.1: आयडी मिळवा आणि कोडमध्ये फाइन-ट्यून केलेल्या मॉडेलची चाचणी करा


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### टप्पा 3.2: फाइन-ट्यून केलेला मॉडेल प्लेग्राउंडमध्ये लोड करा आणि तपासा

आता तुम्ही फाइन-ट्यून केलेल्या मॉडेलची दोन प्रकारे चाचणी करू शकता. पहिले, तुम्ही प्लेग्राउंडला भेट देऊ शकता आणि Models ड्रॉप-डाऊनमधून तुमचे नवीन फाइन-ट्यून केलेले मॉडेल निवडू शकता. दुसरा पर्याय म्हणजे Fine-tuning पॅनलमध्ये दाखवलेला "Playground" पर्याय वापरणे (वरच्या स्क्रीनशॉटमध्ये पहा), ज्यामुळे खालील _तुलनात्मक_ दृश्य उघडते, ज्यात फाउंडेशन आणि फाइन-ट्यून केलेल्या मॉडेलचे व्हर्जन्स एकमेकांच्या शेजारी दाखवले जातात, जेणेकरून पटकन तुलना करता येईल.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.mr.png)

तुमच्या ट्रेनिंग डेटामध्ये वापरलेला सिस्टम संदर्भ (system context) आणि तुमचा चाचणी प्रश्न (test question) येथे भरा. तुम्हाला दिसेल की दोन्ही बाजूंना एकसारखा संदर्भ आणि प्रश्न दाखवला जातो. तुलना चालवा आणि तुम्हाला दोन्हीमधील आउटपुटमधील फरक दिसेल. _लक्षात घ्या की फाइन-ट्यून केलेले मॉडेल उत्तर तुमच्या उदाहरणांमध्ये दिलेल्या फॉरमॅटमध्ये देते, तर फाउंडेशन मॉडेल फक्त सिस्टम प्रॉम्प्टचे पालन करते_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.mr.png)

तुम्हाला दिसेल की तुलना करताना प्रत्येक मॉडेलसाठी टोकन काउंट आणि इनफरन्ससाठी लागलेला वेळ देखील दाखवला जातो. **हा विशिष्ट उदाहरण फक्त प्रक्रिया दाखवण्यासाठी आहे, प्रत्यक्षात हे कोणत्याही खऱ्या डेटासेट किंवा परिस्थितीचे प्रतिनिधित्व करत नाही**. तुम्ही पाहू शकता की दोन्ही उदाहरणांमध्ये टोकनची संख्या सारखीच आहे (कारण सिस्टम संदर्भ आणि युजर प्रॉम्प्ट एकसारखे आहेत), पण फाइन-ट्यून केलेल्या मॉडेलला (कस्टम मॉडेल) इनफरन्ससाठी जास्त वेळ लागतो.

खऱ्या परिस्थितीत, तुम्ही असे साधे उदाहरण वापरणार नाही, तर प्रत्यक्ष डेटावर (उदा. ग्राहक सेवेसाठी प्रॉडक्ट कॅटलॉग) फाइन-ट्यूनिंग कराल, जिथे उत्तराची गुणवत्ता अधिक स्पष्टपणे दिसून येईल. _त्या_ संदर्भात, फाउंडेशन मॉडेलकडून तितक्याच दर्जाचे उत्तर मिळवण्यासाठी अधिक कस्टम प्रॉम्प्ट इंजिनिअरिंग करावे लागेल, ज्यामुळे टोकन वापर वाढेल आणि कदाचित इनफरन्ससाठी लागणारा वेळही वाढेल. _हे स्वतः करून पाहण्यासाठी, OpenAI Cookbook मधील फाइन-ट्यूनिंग उदाहरणे तपासा._



---

**अस्वीकरण**:  
हे दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केले आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानावा. अत्यावश्यक माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थाबद्दल आम्ही जबाबदार राहणार नाही.
